### Import and transformations of API data

In [1]:
if(!is.null(dev.list())) dev.off() # Clear Plots
rm(list=ls()) # Clear objects from Memory
cat("\014") # Clear Console
# writeClipboard(as.character(x)) # copy data frame to clipboard

null device 
          1

In [2]:
library(RCurl)
library(sqldf)
library(digest)
library(dplyr)
library(anytime)
library(geosphere)
library(lubridate)
library(chron)
require(caret)
require(rattle)
require(yardstick)

Warning message:
"package 'RCurl' was built under R version 3.4.4"Loading required package: bitops
Loading required package: gsubfn
Loading required package: proto
Could not load tcltk.  Will use slower R code instead.
Loading required package: RSQLite
Warning message:
"package 'digest' was built under R version 3.4.3"Warning message:
"package 'dplyr' was built under R version 3.4.3"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Warning message:
"package 'anytime' was built under R version 3.4.4"Warning message:
"package 'geosphere' was built under R version 3.4.4"Warning message:
"package 'lubridate' was built under R version 3.4.3"
Attaching package: 'lubridate'

The following object is masked from 'package:base':

    date

Warning message:
"package 'chron' was built under R version 3.4.3"
Attaching package: 'chron'

The following obj

In [3]:
# set working directory
setwd("C:/Users/vanethi/Documents/GitHub/DS420_Factoria")

In [4]:
# set start and end date
startDate <- '2017-12-31-0'
endDate <- '2018-06-01-0'

In [5]:
# pull data for Beijing

# acquire air quality data
bj_aq_url <- paste0("https://biendata.com/competition/airquality/bj/",startDate,"/",endDate,"/2k0d1d8")
bj_aq_file <- getURL(bj_aq_url, ssl.verifyhost=FALSE, ssl.verifypeer=FALSE)
bj_aq_data <- read.csv(textConnection(bj_aq_file), header=TRUE)
  
# acquire API grid meteorology data
bj_gm_url <- paste0("https://biendata.com/competition/meteorology/bj_grid/",startDate,"/",endDate,"/2k0d1d8")
bj_gm_file <- getURL(bj_gm_url, ssl.verifyhost=FALSE, ssl.verifypeer=FALSE)
bj_gm_data <- read.csv(textConnection(bj_gm_file), header=TRUE)

In [6]:
# list of available data frames
df.list <- names(which(unlist(eapply(.GlobalEnv,is.data.frame))))
df.list

[1] "bj_gm_data" "bj_aq_data"

In [7]:
# Converting character to datetime
bj_gm_data$time <- anytime(bj_gm_data$time)
bj_aq_data$time <- anytime(bj_aq_data$time)

In [8]:
# printing structure of all the datasets
for (i in 1:length(df.list)) {
 print(df.list[i])
  print(str(get(df.list[i])))
}

[1] "bj_gm_data"
'data.frame':	911338 obs. of  9 variables:
 $ id            : int  2000958 2000959 2000960 2000961 2000962 2000963 2000964 2000965 2000966 2000967 ...
 $ station_id    : Factor w/ 651 levels "beijing_grid_000",..: 1 2 3 4 5 6 7 8 9 10 ...
 $ time          : POSIXct, format: "2018-03-31 07:00:00" "2018-03-31 07:00:00" ...
 $ weather       : Factor w/ 9 levels "CLEAR_DAY","CLEAR_NIGHT",..: 5 5 5 5 5 5 5 5 5 5 ...
 $ temperature   : num  24 24 25 25 25 25 25 26 22.4 22.4 ...
 $ pressure      : num  973 960 946 931 914 ...
 $ humidity      : num  23 21 19 18 16 15 15 16 16 17 ...
 $ wind_direction: num  161 165 170 184 221 ...
 $ wind_speed    : num  14.28 13.3 12.42 9.84 7.39 ...
NULL
[1] "bj_aq_data"
'data.frame':	47180 obs. of  9 variables:
 $ id                : int  2941450 2941451 2941452 2941453 2941454 2941455 2941456 2941457 2941458 2941459 ...
 $ station_id        : Factor w/ 35 levels "aotizhongxin_aq",..: 7 24 11 27 1 19 26 3 35 10 ...
 $ time              : PO

In [9]:
# Printing the min and max dates of all datasets
print("bj_gm_data")
bj_gm_data %>% summarize(min_date = min(time), max_date = max(time))
print("bj_aq_data")
bj_aq_data %>% summarize(min_date = min(time), max_date = max(time))

[1] "bj_gm_data"


min_date,max_date
2018-03-31 07:00:00,2018-05-29 17:00:00


[1] "bj_aq_data"


min_date,max_date
2018-03-31 07:00:00,2018-05-29 06:00:00


In [10]:
# using only required columns
bj_aq_data <- bj_aq_data %>% select(-id)
bj_gm_data <- bj_gm_data %>% select(-c(id, weather))

In [11]:
# Beijing closest grids to stations
bj_closest_stations <- read.csv('SL_beijing_closest_stations.csv')
colnames(bj_closest_stations) <- c('x',"stationId","stationName","distance") 

In [12]:
# Modifying column names for consistency
colnames(bj_aq_data) <- c("stationId","utc_time","PM2.5","PM10","NO2","CO","O3", "SO2") 
colnames(bj_gm_data) <- c("stationName","utc_time","temperature","pressure","humidity","wind_direction","wind_speed.kph") 

In [13]:
str(bj_closest_stations)
str(bj_gm_data)
str(bj_aq_data)
head(bj_closest_stations)

'data.frame':	35 obs. of  4 variables:
 $ x          : int  1 2 3 4 5 6 7 8 9 10 ...
 $ stationId  : Factor w/ 35 levels "aotizhongxin_aq",..: 7 24 11 27 1 19 26 3 35 10 ...
 $ stationName: Factor w/ 26 levels "beijing_grid_216",..: 16 16 13 16 17 19 14 9 8 13 ...
 $ distance   : num  3540 1669 4638 4776 2020 ...
'data.frame':	911338 obs. of  7 variables:
 $ stationName   : Factor w/ 651 levels "beijing_grid_000",..: 1 2 3 4 5 6 7 8 9 10 ...
 $ utc_time      : POSIXct, format: "2018-03-31 07:00:00" "2018-03-31 07:00:00" ...
 $ temperature   : num  24 24 25 25 25 25 25 26 22.4 22.4 ...
 $ pressure      : num  973 960 946 931 914 ...
 $ humidity      : num  23 21 19 18 16 15 15 16 16 17 ...
 $ wind_direction: num  161 165 170 184 221 ...
 $ wind_speed.kph: num  14.28 13.3 12.42 9.84 7.39 ...
'data.frame':	47180 obs. of  8 variables:
 $ stationId: Factor w/ 35 levels "aotizhongxin_aq",..: 7 24 11 27 1 19 26 3 35 10 ...
 $ utc_time : POSIXct, format: "2018-03-31 07:00:00" "2018-03-31 07:00

x,stationId,stationName,distance
1,dongsi_aq,beijing_grid_303,3539.569
2,tiantan_aq,beijing_grid_303,1669.215
3,guanyuan_aq,beijing_grid_282,4637.889
4,wanshouxigong_aq,beijing_grid_303,4775.641
5,aotizhongxin_aq,beijing_grid_304,2020.020
6,nongzhanguan_aq,beijing_grid_324,5296.386


In [14]:
# Mapping stationIds with respective grids
bj_aq_map <- merge(bj_aq_data,bj_closest_stations, by = "stationId")

In [15]:
# Merge of AirQuality and Meteorology data
bj_aq_gm_data <- merge(bj_aq_map, bj_gm_data, by = c("stationName","utc_time"))

In [16]:
str(bj_aq_gm_data)
head(bj_aq_gm_data)
bj_aq_gm_data %>% summarize(min_date = min(utc_time), max_date = max(utc_time))

'data.frame':	46372 obs. of  16 variables:
 $ stationName   : Factor w/ 26 levels "beijing_grid_216",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ utc_time      : POSIXct, format: "2018-03-31 07:00:00" "2018-03-31 08:00:00" ...
 $ stationId     : Factor w/ 35 levels "aotizhongxin_aq",..: 14 14 14 14 14 14 14 14 14 14 ...
 $ PM2.5         : num  105 120 121 137 146 270 160 180 136 149 ...
 $ PM10          : num  163 185 185 212 254 323 297 280 204 219 ...
 $ NO2           : num  45 52 55 59 68 59 85 76 61 87 ...
 $ CO            : num  0.8 0.9 0.9 1 1 1.1 1.1 1.1 1.1 1.2 ...
 $ O3            : num  137 140 141 132 106 96 46 45 50 7 ...
 $ SO2           : num  6 NA NA NA NA 20 8 NA NA NA ...
 $ x             : int  30 30 30 30 30 30 30 30 30 30 ...
 $ distance      : num  2226 2226 2226 2226 2226 ...
 $ temperature   : num  22 23 23 24 20 19 17 13 14 12.4 ...
 $ pressure      : num  1001 1000 1000 999 1000 ...
 $ humidity      : num  20 20 22 24 28 31 32 32 31 32 ...
 $ wind_direction: num  159 174 188

stationName,utc_time,stationId,PM2.5,PM10,NO2,CO,O3,SO2,x,distance,temperature,pressure,humidity,wind_direction,wind_speed.kph
beijing_grid_216,2018-03-31 07:00:00,liulihe_aq,105,163,45,0.8,137,6,30,2226.39,22,1000.8969,20,159.45,10.63
beijing_grid_216,2018-03-31 08:00:00,liulihe_aq,120,185,52,0.9,140,NA,30,2226.39,23,1000.0580,20,173.73,10.96
beijing_grid_216,2018-03-31 09:00:00,liulihe_aq,121,185,55,0.9,141,NA,30,2226.39,23,999.5212,22,188.34,10.21
beijing_grid_216,2018-03-31 10:00:00,liulihe_aq,137,212,59,1.0,132,NA,30,2226.39,24,999.4476,24,204.63,9.83
beijing_grid_216,2018-03-31 11:00:00,liulihe_aq,146,254,68,1.0,106,NA,30,2226.39,20,999.6694,28,232.00,7.27
beijing_grid_216,2018-03-31 12:00:00,liulihe_aq,270,323,59,1.1,96,20,30,2226.39,19,1000.0671,31,268.92,6.59


min_date,max_date
2018-03-31 07:00:00,2018-05-29 06:00:00


In [17]:
rm("bj_gm_data")
rm("bj_aq_data")
rm("bj_aq_map")
rm("bj_closest_stations")

In [18]:
# selecting only required columns
bj_aq_gm_data <- bj_aq_gm_data %>% select(c("stationName","utc_time","stationId","PM2.5","PM10", "O3",
                                            "temperature","pressure","humidity","wind_direction","wind_speed.kph"))

In [19]:
# adding additional columns for modeling
bj_aq_gm_data$hour <- hour(bj_aq_gm_data$utc_time)
bj_aq_gm_data$month <- month(bj_aq_gm_data$utc_time)
bj_aq_gm_data$date <- date(bj_aq_gm_data$utc_time)
bj_aq_gm_data$weekend = chron::is.weekend(bj_aq_gm_data$date)

In [20]:
str(bj_aq_gm_data)

'data.frame':	46372 obs. of  15 variables:
 $ stationName   : Factor w/ 26 levels "beijing_grid_216",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ utc_time      : POSIXct, format: "2018-03-31 07:00:00" "2018-03-31 08:00:00" ...
 $ stationId     : Factor w/ 35 levels "aotizhongxin_aq",..: 14 14 14 14 14 14 14 14 14 14 ...
 $ PM2.5         : num  105 120 121 137 146 270 160 180 136 149 ...
 $ PM10          : num  163 185 185 212 254 323 297 280 204 219 ...
 $ O3            : num  137 140 141 132 106 96 46 45 50 7 ...
 $ temperature   : num  22 23 23 24 20 19 17 13 14 12.4 ...
 $ pressure      : num  1001 1000 1000 999 1000 ...
 $ humidity      : num  20 20 22 24 28 31 32 32 31 32 ...
 $ wind_direction: num  159 174 188 205 232 ...
 $ wind_speed.kph: num  10.63 10.96 10.21 9.83 7.27 ...
 $ hour          : int  7 8 9 10 11 12 13 14 15 17 ...
 $ month         : num  3 3 3 3 3 3 3 3 3 3 ...
 $ date          : Date, format: "2018-03-31" "2018-03-31" ...
 $ weekend       : logi  TRUE TRUE TRUE TRUE TRUE TRUE

### Import and transformations of Historical data

In [21]:
# retreiving historical files
bj_aq_gm_hist_file <- "Ready for Modeling/bj_aq_gm_hist_data.csv"
bj_aq_gm_hist_data <- read.csv(bj_aq_gm_hist_file, header=TRUE, sep=",", stringsAsFactors = FALSE)

In [22]:
# data transformations for consistency
bj_aq_gm_hist_data <- bj_aq_gm_hist_data %>% select(-X)
bj_aq_gm_hist_data$utc_time <- anytime(bj_aq_gm_hist_data$utc_time)
bj_aq_gm_hist_data$date <- as.Date(bj_aq_gm_hist_data$date , "%Y-%m-%d")

In [23]:
str(bj_aq_gm_hist_data)

'data.frame':	356825 obs. of  15 variables:
 $ stationName   : chr  "beijing_grid_216" "beijing_grid_216" "beijing_grid_216" "beijing_grid_216" ...
 $ utc_time      : POSIXct, format: "2017-01-01 14:00:00" "2017-01-01 15:00:00" ...
 $ stationId     : chr  "liulihe_aq" "liulihe_aq" "liulihe_aq" "liulihe_aq" ...
 $ PM2.5         : int  376 369 361 354 356 315 287 254 231 224 ...
 $ PM10          : int  447 407 389 NA 360 NA NA NA NA NA ...
 $ O3            : int  2 2 2 2 2 2 NA 2 2 2 ...
 $ temperature   : num  -2.22 -2.37 -2.58 -2.79 -2.99 -3.3 -3.6 -3.9 -4.11 -4.33 ...
 $ pressure      : num  1015 1015 1015 1015 1015 ...
 $ humidity      : num  62.9 65.8 65.3 64.9 64.5 ...
 $ wind_direction: num  284 298 308 319 330 ...
 $ wind_speed.kph: num  4.28 4.39 4.19 4.14 4.24 4.1 3.99 3.92 2.97 2.25 ...
 $ hour          : int  14 15 16 17 18 19 20 21 22 23 ...
 $ month         : int  1 1 1 1 1 1 1 1 1 1 ...
 $ date          : Date, format: "2017-01-01" "2017-01-01" ...
 $ weekend       : logi 

In [24]:
# Append API and hist data
bj_aq_gm_combined_data <- rbind(bj_aq_gm_data, bj_aq_gm_hist_data)

In [25]:
str(bj_aq_gm_combined_data)
bj_aq_gm_combined_data %>% summarize(min_date = min(date), max_date = max(date))
summary(bj_aq_gm_combined_data)

'data.frame':	403197 obs. of  15 variables:
 $ stationName   : Factor w/ 26 levels "beijing_grid_216",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ utc_time      : POSIXct, format: "2018-03-31 07:00:00" "2018-03-31 08:00:00" ...
 $ stationId     : Factor w/ 35 levels "aotizhongxin_aq",..: 14 14 14 14 14 14 14 14 14 14 ...
 $ PM2.5         : num  105 120 121 137 146 270 160 180 136 149 ...
 $ PM10          : num  163 185 185 212 254 323 297 280 204 219 ...
 $ O3            : num  137 140 141 132 106 96 46 45 50 7 ...
 $ temperature   : num  22 23 23 24 20 19 17 13 14 12.4 ...
 $ pressure      : num  1001 1000 1000 999 1000 ...
 $ humidity      : num  20 20 22 24 28 31 32 32 31 32 ...
 $ wind_direction: num  159 174 188 205 232 ...
 $ wind_speed.kph: num  10.63 10.96 10.21 9.83 7.27 ...
 $ hour          : int  7 8 9 10 11 12 13 14 15 17 ...
 $ month         : num  3 3 3 3 3 3 3 3 3 3 ...
 $ date          : Date, format: "2018-03-31" "2018-03-31" ...
 $ weekend       : logi  TRUE TRUE TRUE TRUE TRUE TRU

min_date,max_date
2017-01-01,2018-05-29


           stationName        utc_time                           stationId     
 beijing_grid_303: 69120   Min.   :2017-01-01 14:00:00   badaling_aq  : 11520  
 beijing_grid_282: 23040   1st Qu.:2017-05-10 00:00:00   beibuxinqu_aq: 11520  
 beijing_grid_283: 23040   Median :2017-09-24 18:00:00   daxing_aq    : 11520  
 beijing_grid_324: 23040   Mean   :2017-09-19 03:55:07   dingling_aq  : 11520  
 beijing_grid_452: 23040   3rd Qu.:2018-01-21 08:00:00   donggaocun_aq: 11520  
 beijing_grid_216: 11520   Max.   :2018-05-29 06:00:00   dongsi_aq    : 11520  
 (Other)         :230397                                 (Other)      :334077  
     PM2.5              PM10               O3         temperature    
 Min.   :   2.00   Min.   :   5.00   Min.   :  1.0   Min.   :-18.37  
 1st Qu.:  17.00   1st Qu.:  40.00   1st Qu.: 15.0   1st Qu.:  0.55  
 Median :  41.00   Median :  75.00   Median : 49.0   Median : 11.72  
 Mean   :  60.77   Mean   :  93.86   Mean   : 58.4   Mean   : 11.41  
 3rd Qu.: 

In [26]:
beijing_holidays <- c("2017-01-01", "2017-01-02","2017-01-27", "2017-01-28", "2017-01-29","2017-01-30", "2017-01-31"
                      , "2017-02-01", "2017-02-02", "2017-04-02", "2017-04-03","2017-04-04", "2017-05-01", "2017-05-28"
                      , "2017-05-29", "2017-05-30", "2017-10-01", "2017-10-02", "2017-10-03", "2017-10-04", "2017-10-05"
                      , "2017-10-06", "2017-10-07", "2017-10-08", "2018-01-01", "2018-02-15", "2018-02-16", "2018-02-17"
                      , "2018-02-18", "2018-02-19", "2018-02-20", "2018-02-21", "2018-04-05", "2018-04-06", "2018-04-07"
                      , "2018-04-30", "2018-05-01")

In [27]:
bj_aq_gm_combined_data$holiday <- ifelse(bj_aq_gm_combined_data$date %in% as.Date(beijing_holidays), 1, 0)

In [28]:
rm("bj_aq_gm_data")
rm("bj_aq_gm_hist_data")

In [29]:
#code to replace outliers with NA
bj_aq_gm_combined_data[bj_aq_gm_combined_data$PM2.5 %in% boxplot.stats(bj_aq_gm_combined_data$PM2.5)$out, ]$PM2.5 <- NA
bj_aq_gm_combined_data[bj_aq_gm_combined_data$PM10 %in% boxplot.stats(bj_aq_gm_combined_data$PM10)$out, ]$PM10 <- NA
bj_aq_gm_combined_data[bj_aq_gm_combined_data$O3 %in% boxplot.stats(bj_aq_gm_combined_data$O3)$out, ]$O3 <- NA

In [30]:
str(bj_aq_gm_combined_data)
summary(bj_aq_gm_combined_data)

'data.frame':	403197 obs. of  16 variables:
 $ stationName   : Factor w/ 26 levels "beijing_grid_216",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ utc_time      : POSIXct, format: "2018-03-31 07:00:00" "2018-03-31 08:00:00" ...
 $ stationId     : Factor w/ 35 levels "aotizhongxin_aq",..: 14 14 14 14 14 14 14 14 14 14 ...
 $ PM2.5         : num  105 120 121 137 146 NA 160 NA 136 149 ...
 $ PM10          : num  163 185 185 212 NA NA NA NA 204 219 ...
 $ O3            : num  137 140 141 132 106 96 46 45 50 7 ...
 $ temperature   : num  22 23 23 24 20 19 17 13 14 12.4 ...
 $ pressure      : num  1001 1000 1000 999 1000 ...
 $ humidity      : num  20 20 22 24 28 31 32 32 31 32 ...
 $ wind_direction: num  159 174 188 205 232 ...
 $ wind_speed.kph: num  10.63 10.96 10.21 9.83 7.27 ...
 $ hour          : int  7 8 9 10 11 12 13 14 15 17 ...
 $ month         : num  3 3 3 3 3 3 3 3 3 3 ...
 $ date          : Date, format: "2018-03-31" "2018-03-31" ...
 $ weekend       : logi  TRUE TRUE TRUE TRUE TRUE TRUE ...


           stationName        utc_time                           stationId     
 beijing_grid_303: 69120   Min.   :2017-01-01 14:00:00   badaling_aq  : 11520  
 beijing_grid_282: 23040   1st Qu.:2017-05-10 00:00:00   beibuxinqu_aq: 11520  
 beijing_grid_283: 23040   Median :2017-09-24 18:00:00   daxing_aq    : 11520  
 beijing_grid_324: 23040   Mean   :2017-09-19 03:55:07   dingling_aq  : 11520  
 beijing_grid_452: 23040   3rd Qu.:2018-01-21 08:00:00   donggaocun_aq: 11520  
 beijing_grid_216: 11520   Max.   :2018-05-29 06:00:00   dongsi_aq    : 11520  
 (Other)         :230397                                 (Other)      :334077  
     PM2.5             PM10              O3          temperature    
 Min.   :  2.00   Min.   :  5.00   Min.   :  1.00   Min.   :-18.37  
 1st Qu.: 16.00   1st Qu.: 39.00   1st Qu.: 14.00   1st Qu.:  0.55  
 Median : 38.00   Median : 72.00   Median : 47.00   Median : 11.72  
 Mean   : 49.67   Mean   : 82.39   Mean   : 52.57   Mean   : 11.41  
 3rd Qu.: 72.00

In [31]:
# replace PM2.5 NA with mean values based on certain groups
bj_aq_gm_combined_data_PM2.5 <- bj_aq_gm_combined_data %>% 
                                group_by(stationId, hour, month, weekend, holiday) %>% 
                                summarize(PM2.5 = mean(PM2.5, na.rm = TRUE))
bj_aq_gm_combined_data <- bj_aq_gm_combined_data %>% 
                          left_join(bj_aq_gm_combined_data_PM2.5, c("stationId", "hour", "month", "weekend", "holiday")) %>% 
                          mutate(PM2.5 = coalesce(PM2.5.x, PM2.5.y)) %>% 
                          select(-PM2.5.x, -PM2.5.y)
bj_aq_gm_combined_data_PM2.5 <- bj_aq_gm_combined_data %>% 
                                group_by(stationId, hour, weekend, holiday) %>% 
                                summarize(PM2.5 = mean(PM2.5, na.rm = TRUE))
bj_aq_gm_combined_data <- bj_aq_gm_combined_data %>% 
                          left_join(bj_aq_gm_combined_data_PM2.5, c("stationId", "hour", "weekend","holiday")) %>% 
                          mutate(PM2.5 = coalesce(PM2.5.x, PM2.5.y)) %>% 
                          select(-PM2.5.x, -PM2.5.y)

Warning message:
"package 'bindrcpp' was built under R version 3.4.3"

In [32]:
# replace PM10 NA with mean values based on certain groups
bj_aq_gm_combined_data_PM10 <- bj_aq_gm_combined_data %>% 
                                group_by(stationId, hour, month, weekend, holiday) %>% 
                                summarize(PM10 = mean(PM10, na.rm = TRUE))
bj_aq_gm_combined_data <- bj_aq_gm_combined_data %>% 
                          left_join(bj_aq_gm_combined_data_PM10, c("stationId", "hour", "month", "weekend", "holiday")) %>% 
                          mutate(PM10 = coalesce(PM10.x, PM10.y)) %>% 
                          select(-PM10.x, -PM10.y)
bj_aq_gm_combined_data_PM10 <- bj_aq_gm_combined_data %>% 
                                group_by(stationId, hour, weekend, holiday) %>% 
                                summarize(PM10 = mean(PM10, na.rm = TRUE))
bj_aq_gm_combined_data <- bj_aq_gm_combined_data %>% 
                          left_join(bj_aq_gm_combined_data_PM10, c("stationId", "hour", "weekend", "holiday")) %>% 
                          mutate(PM10 = coalesce(PM10.x, PM10.y)) %>% 
                          select(-PM10.x, -PM10.y)

In [33]:
# replace O3 NA with mean values based on certain groups
bj_aq_gm_combined_data_O3 <- bj_aq_gm_combined_data %>% 
                                group_by(stationId, hour, month, weekend, holiday) %>% 
                                summarize(O3 = mean(O3, na.rm = TRUE))
bj_aq_gm_combined_data <- bj_aq_gm_combined_data %>% 
                          left_join(bj_aq_gm_combined_data_O3, c("stationId", "hour", "month", "weekend", "holiday")) %>% 
                          mutate(O3 = coalesce(O3.x, O3.y)) %>% 
                          select(-O3.x, -O3.y)
bj_aq_gm_combined_data_O3 <- bj_aq_gm_combined_data %>% 
                                group_by(stationId, hour,  weekend, holiday) %>% 
                                summarize(O3 = mean(O3, na.rm = TRUE))
bj_aq_gm_combined_data <- bj_aq_gm_combined_data %>% 
                          left_join(bj_aq_gm_combined_data_O3, c("stationId", "hour",  "weekend", "holiday")) %>% 
                          mutate(O3 = coalesce(O3.x, O3.y)) %>% 
                          select(-O3.x, -O3.y)

In [34]:
rm("bj_aq_gm_combined_data_PM2.5")
rm("bj_aq_gm_combined_data_PM10")
rm("bj_aq_gm_combined_data_O3")

In [35]:
str(bj_aq_gm_combined_data)
summary(bj_aq_gm_combined_data)

'data.frame':	403197 obs. of  16 variables:
 $ stationName   : Factor w/ 26 levels "beijing_grid_216",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ utc_time      : POSIXct, format: "2018-03-31 07:00:00" "2018-03-31 08:00:00" ...
 $ stationId     : Factor w/ 35 levels "aotizhongxin_aq",..: 14 14 14 14 14 14 14 14 14 14 ...
 $ temperature   : num  22 23 23 24 20 19 17 13 14 12.4 ...
 $ pressure      : num  1001 1000 1000 999 1000 ...
 $ humidity      : num  20 20 22 24 28 31 32 32 31 32 ...
 $ wind_direction: num  159 174 188 205 232 ...
 $ wind_speed.kph: num  10.63 10.96 10.21 9.83 7.27 ...
 $ hour          : int  7 8 9 10 11 12 13 14 15 17 ...
 $ month         : num  3 3 3 3 3 3 3 3 3 3 ...
 $ date          : Date, format: "2018-03-31" "2018-03-31" ...
 $ weekend       : logi  TRUE TRUE TRUE TRUE TRUE TRUE ...
 $ holiday       : num  0 0 0 0 0 0 0 0 0 0 ...
 $ PM2.5         : num  105 120 121 137 146 ...
 $ PM10          : num  163 185 185 212 116 ...
 $ O3            : num  137 140 141 132 106 96 4

           stationName        utc_time                           stationId     
 beijing_grid_303: 69120   Min.   :2017-01-01 14:00:00   badaling_aq  : 11520  
 beijing_grid_282: 23040   1st Qu.:2017-05-10 00:00:00   beibuxinqu_aq: 11520  
 beijing_grid_283: 23040   Median :2017-09-24 18:00:00   daxing_aq    : 11520  
 beijing_grid_324: 23040   Mean   :2017-09-19 03:55:07   dingling_aq  : 11520  
 beijing_grid_452: 23040   3rd Qu.:2018-01-21 08:00:00   donggaocun_aq: 11520  
 beijing_grid_216: 11520   Max.   :2018-05-29 06:00:00   dongsi_aq    : 11520  
 (Other)         :230397                                 (Other)      :334077  
  temperature        pressure         humidity      wind_direction 
 Min.   :-18.37   Min.   : 917.6   Min.   :  4.59   Min.   :  0.0  
 1st Qu.:  0.55   1st Qu.: 993.2   1st Qu.: 21.65   1st Qu.:106.7  
 Median : 11.72   Median :1004.4   Median : 33.01   Median :189.1  
 Mean   : 11.41   Mean   :1001.2   Mean   : 37.93   Mean   :191.0  
 3rd Qu.: 22.00   3r

In [36]:
# validating for no NAs
bj_aq_gm_combined_data[!complete.cases(bj_aq_gm_combined_data),]

stationName,utc_time,stationId,temperature,pressure,humidity,wind_direction,wind_speed.kph,hour,month,date,weekend,holiday,PM2.5,PM10,O3


In [37]:
# converting logical weekend value to numeric for modeling
bj_aq_gm_combined_data$weekend <- as.integer(bj_aq_gm_combined_data$weekend)

In [38]:
# selecting only required columns
bj_aq_gm_combined_data <- 
bj_aq_gm_combined_data %>% select(c("stationName","utc_time","stationId","PM2.5","PM10", "O3",
                                    "temperature","pressure","humidity","wind_direction","wind_speed.kph", 
                                    "hour", "month", "date", "weekend", "holiday"))

In [39]:
# retreiving lat long data  
bj_lat_long_file <- "Datasets/Beijing_AirQuality_Stations.csv"
bj_lat_long_data <- read.csv(bj_lat_long_file, header=TRUE, sep=",", stringsAsFactors = FALSE) 
colnames(bj_lat_long_data) <- c("stationId", "longitude", "latitude")

In [40]:
bj_aq_gm_combined_data <- merge(bj_aq_gm_combined_data,bj_lat_long_data, by = "stationId")

In [41]:
str(bj_aq_gm_combined_data)
bj_aq_gm_combined_data %>% summarize(min_date = min(utc_time), max_date = max(utc_time))
summary(bj_aq_gm_combined_data)

'data.frame':	403197 obs. of  18 variables:
 $ stationId     : Factor w/ 35 levels "aotizhongxin_aq",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ stationName   : Factor w/ 26 levels "beijing_grid_216",..: 17 17 17 17 17 17 17 17 17 17 ...
 $ utc_time      : POSIXct, format: "2017-01-06 17:00:00" "2017-01-06 18:00:00" ...
 $ PM2.5         : num  44.1 49.2 46 43.3 37.5 ...
 $ PM10          : num  61.3 64.4 61.4 56.6 50.8 ...
 $ O3            : num  2 2 2 33.1 2 ...
 $ temperature   : num  -3.67 -3.86 -3.9 -3.94 -3.98 -3.81 -3.63 -3.46 -3.4 -3.7 ...
 $ pressure      : num  1019 1018 1018 1018 1018 ...
 $ humidity      : num  87.2 87.4 87.5 87.5 87.5 ...
 $ wind_direction: num  65.58 47.84 30.07 15.64 4.96 ...
 $ wind_speed.kph: num  1.87 1.71 1.83 2.09 2.46 2.61 2.8 3.01 5.15 5.03 ...
 $ hour          : int  17 18 19 20 21 22 23 0 17 18 ...
 $ month         : num  1 1 1 1 1 1 1 1 1 1 ...
 $ date          : Date, format: "2017-01-06" "2017-01-06" ...
 $ weekend       : int  0 0 0 0 0 0 0 1 0 0 ...
 $ ho

min_date,max_date
2017-01-01 14:00:00,2018-05-29 06:00:00


         stationId                stationName        utc_time                  
 badaling_aq  : 11520   beijing_grid_303: 69120   Min.   :2017-01-01 14:00:00  
 beibuxinqu_aq: 11520   beijing_grid_282: 23040   1st Qu.:2017-05-10 00:00:00  
 daxing_aq    : 11520   beijing_grid_283: 23040   Median :2017-09-24 18:00:00  
 dingling_aq  : 11520   beijing_grid_324: 23040   Mean   :2017-09-19 03:55:07  
 donggaocun_aq: 11520   beijing_grid_452: 23040   3rd Qu.:2018-01-21 08:00:00  
 dongsi_aq    : 11520   beijing_grid_216: 11520   Max.   :2018-05-29 06:00:00  
 (Other)      :334077   (Other)         :230397                                
     PM2.5             PM10              O3          temperature    
 Min.   :  2.00   Min.   :  5.00   Min.   :  1.00   Min.   :-18.37  
 1st Qu.: 18.00   1st Qu.: 45.00   1st Qu.: 16.00   1st Qu.:  0.55  
 Median : 41.00   Median : 73.16   Median : 49.00   Median : 11.72  
 Mean   : 49.72   Mean   : 80.84   Mean   : 54.75   Mean   : 11.41  
 3rd Qu.: 69.00

### Model Training and validation

In [42]:
set.seed(2306)

In [43]:
# Training and test data set partition
sample_size <- floor(0.8 * nrow(bj_aq_gm_combined_data))
train_index <- sample(seq_len(nrow(bj_aq_gm_combined_data)), size = sample_size)
train_bj <- bj_aq_gm_combined_data[train_index, ]
test_bj <- bj_aq_gm_combined_data[-train_index, ]

In [44]:
str(train_bj)
str(test_bj)

'data.frame':	322557 obs. of  18 variables:
 $ stationId     : Factor w/ 35 levels "aotizhongxin_aq",..: 11 22 31 21 14 4 3 14 29 11 ...
 $ stationName   : Factor w/ 26 levels "beijing_grid_216",..: 13 16 16 26 1 15 9 1 3 13 ...
 $ utc_time      : POSIXct, format: "2017-12-16 02:00:00" "2017-10-13 20:00:00" ...
 $ PM2.5         : num  7 13 38 37.4 38 ...
 $ PM10          : num  39 13 44 34.5 117 ...
 $ O3            : num  59 2.5 28 19 48 6 2 51 85 27 ...
 $ temperature   : num  -3.32 12.63 -0.89 3.65 16 ...
 $ pressure      : num  1030 1021 1022 1009 1010 ...
 $ humidity      : num  17.2 26.8 58 41 29 ...
 $ wind_direction: num  319.7 18.6 270.1 117.2 265.2 ...
 $ wind_speed.kph: num  21.77 7.4 9.04 6.17 5.93 ...
 $ hour          : int  2 20 13 7 18 18 17 4 22 13 ...
 $ month         : num  12 10 2 1 5 2 4 4 2 2 ...
 $ date          : Date, format: "2017-12-16" "2017-10-13" ...
 $ weekend       : int  1 0 0 0 0 0 0 1 0 1 ...
 $ holiday       : num  0 0 0 0 0 0 0 0 0 1 ...
 $ longitude

In [45]:
# Predicting PM2.5 using stationId, hour, month and weekend variables
PM2.5_bj_formula <- as.formula("PM2.5 ~ latitude + longitude + hour + month + weekend + holiday")
PM2.5_bj_model <- train(PM2.5_bj_formula, data = train_bj, method = "lm" )
summary(PM2.5_bj_model)
test_bj$PM2.5_pred <- predict(PM2.5_bj_model, test_bj)
metrics(test_bj, truth = PM2.5, estimate = PM2.5_pred)


Call:
lm(formula = .outcome ~ ., data = dat)

Residuals:
    Min      1Q  Median      3Q     Max 
-62.596 -30.775  -8.657  19.014 138.545 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) 588.934825  28.928809  20.358  < 2e-16 ***
latitude    -15.980988   0.303843 -52.596  < 2e-16 ***
longitude     0.872005   0.248319   3.512 0.000445 ***
hour          0.050710   0.009912   5.116 3.12e-07 ***
month        -0.521569   0.019657 -26.534  < 2e-16 ***
weekend       0.867280   0.151340   5.731 1.00e-08 ***
holiday       5.455431   0.269086  20.274  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 38.92 on 322550 degrees of freedom
Multiple R-squared:  0.0126,	Adjusted R-squared:  0.01259 
F-statistic: 686.2 on 6 and 322550 DF,  p-value: < 2.2e-16


rmse,rsq
39.0058,0.01328572


In [46]:
# Predicting PM10 using stationId, hour, month, weekend variables along with previously predicted PM2.5 
PM10_bj_formula <- as.formula("PM10 ~ PM2.5 + latitude + longitude + hour + month + weekend + holiday")
PM10_bj_model <- train(PM10_bj_formula, data = train_bj, method = "lm" )
summary(PM10_bj_model)
test_bj$PM10_pred <- predict(PM10_bj_model, test_bj)
metrics(test_bj, truth = PM10, estimate = PM10_pred)


Call:
lm(formula = .outcome ~ ., data = dat)

Residuals:
     Min       1Q   Median       3Q      Max 
-154.880  -23.436   -4.927   18.000  199.188 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  2.570e+03  2.834e+01  90.687  < 2e-16 ***
PM2.5        7.025e-01  1.724e-03 407.508  < 2e-16 ***
latitude    -1.919e+01  2.987e-01 -64.233  < 2e-16 ***
longitude   -1.506e+01  2.431e-01 -61.964  < 2e-16 ***
hour        -5.686e-02  9.705e-03  -5.859 4.67e-09 ***
month       -5.750e-01  1.927e-02 -29.849  < 2e-16 ***
weekend      3.774e+00  1.482e-01  25.471  < 2e-16 ***
holiday     -7.537e+00  2.636e-01 -28.592  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 38.1 on 322549 degrees of freedom
Multiple R-squared:  0.3666,	Adjusted R-squared:  0.3666 
F-statistic: 2.667e+04 on 7 and 322549 DF,  p-value: < 2.2e-16


rmse,rsq
38.11017,0.3714401


In [47]:
# Predicting O3 using stationId, hour, month, weekend variables along with previously predicted PM2.5, PM10 and O3 variables
O3_bj_formula <- as.formula("O3 ~ PM2.5 + PM10 + latitude + longitude + hour + month + weekend + holiday")
O3_bj_model <- train(O3_bj_formula, data = train_bj, method = "lm" )
summary(O3_bj_model)
test_bj$O3_pred <- predict(O3_bj_model, test_bj)
metrics(test_bj, truth = O3, estimate = O3_pred)


Call:
lm(formula = .outcome ~ ., data = dat)

Residuals:
    Min      1Q  Median      3Q     Max 
-94.831 -31.583  -6.615  24.283 151.120 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) -1.571e+03  3.103e+01  -50.64  < 2e-16 ***
PM2.5       -1.585e-01  2.294e-03  -69.12  < 2e-16 ***
PM10         8.682e-02  1.903e-03   45.61  < 2e-16 ***
latitude     1.826e+01  3.250e-01   56.20  < 2e-16 ***
longitude    7.919e+00  2.644e-01   29.95  < 2e-16 ***
hour        -1.726e+00  1.049e-02 -164.53  < 2e-16 ***
month       -8.666e-01  2.086e-02  -41.55  < 2e-16 ***
weekend     -4.971e-01  1.603e-01   -3.10  0.00193 ** 
holiday     -6.952e+00  2.853e-01  -24.37  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 41.19 on 322548 degrees of freedom
Multiple R-squared:  0.1083,	Adjusted R-squared:  0.1083 
F-statistic:  4896 on 8 and 322548 DF,  p-value: < 2.2e-16


rmse,rsq
41.35863,0.1047347


In [48]:
rm("train_bj")
rm("test_bj")

### Building the next 2 days dataset and predicting PM2.5, PM10 and O3 values

In [49]:
#setting system timezone to UTC for consistent datetime usage
Sys.setenv(TZ='GMT')
tomorrow <- Sys.Date() + 1

In [50]:
# building 1 hour intervals for next 2 days and stationIds
bj_time <-seq(from= as.POSIXct(tomorrow), by = "1 hour", length.out = 48)
bj_time <- with_tz(bj_time, tzone = "UTC")
bj_future_data <- data.frame(bj_time)
# This id will be used in the creation of final submission file
bj_future_data$id <- seq.int(nrow(bj_future_data)) -1
bj_future_data <- merge(bj_future_data, data.frame(unique(bj_aq_gm_combined_data$stationId)))
names(bj_future_data) <- c("utc_time", "id", "stationId")

In [51]:
# building datetime features for the next 2 days
bj_future_data$hour <- hour(bj_future_data$utc_time)
bj_future_data$month <- month(bj_future_data$utc_time)
bj_future_data$date <- date(bj_future_data$utc_time)
bj_future_data$weekend = chron::is.weekend(bj_future_data$date)

In [52]:
bj_future_data$weekend <- as.integer(bj_future_data$weekend)
bj_future_data$holiday <- ifelse(bj_future_data$date %in% as.Date(beijing_holidays), 1, 0)

In [53]:
#lat long data
bj_future_data <- merge(bj_future_data,bj_lat_long_data, by = "stationId")

In [54]:
# predicting PM2.5, PM10 and O3 values
bj_future_data$PM2.5 <- predict(PM2.5_bj_model, bj_future_data)
bj_future_data$PM10 <- predict(PM10_bj_model, bj_future_data)
bj_future_data$O3 <- predict(O3_bj_model, bj_future_data)

In [55]:
#creating the test_id for final submission file
bj_future_data$test_id <- paste(bj_future_data$stationId, "#", bj_future_data$id, sep = "")

In [56]:
str(bj_future_data)
summary(bj_future_data)
bj_future_data %>% summarize(min_date = min(utc_time ), max_date = max(utc_time ))

'data.frame':	1680 obs. of  14 variables:
 $ stationId: Factor w/ 35 levels "aotizhongxin_aq",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ utc_time : POSIXct, format: "2018-05-30 00:00:00" "2018-05-30 01:00:00" ...
 $ id       : num  0 1 2 3 4 5 6 7 8 9 ...
 $ hour     : int  0 1 2 3 4 5 6 7 8 9 ...
 $ month    : num  5 5 5 5 5 5 5 5 5 5 ...
 $ date     : Date, format: "2018-05-30" "2018-05-30" ...
 $ weekend  : int  0 0 0 0 0 0 0 0 0 0 ...
 $ holiday  : num  0 0 0 0 0 0 0 0 0 0 ...
 $ longitude: num  116 116 116 116 116 ...
 $ latitude : num  40 40 40 40 40 ...
 $ PM2.5    : num  48.9 48.9 49 49 49.1 ...
 $ PM10     : num  80.9 80.9 80.8 80.8 80.8 ...
 $ O3       : num  75.7 73.9 72.2 70.5 68.7 ...
 $ test_id  : chr  "aotizhongxin_aq#0" "aotizhongxin_aq#1" "aotizhongxin_aq#2" "aotizhongxin_aq#3" ...


           stationId       utc_time                         id       
 aotizhongxin_aq:  48   Min.   :2018-05-30 00:00:00   Min.   : 0.00  
 badaling_aq    :  48   1st Qu.:2018-05-30 11:45:00   1st Qu.:11.75  
 beibuxinqu_aq  :  48   Median :2018-05-30 23:30:00   Median :23.50  
 daxing_aq      :  48   Mean   :2018-05-30 23:30:00   Mean   :23.50  
 dingling_aq    :  48   3rd Qu.:2018-05-31 11:15:00   3rd Qu.:35.25  
 donggaocun_aq  :  48   Max.   :2018-05-31 23:00:00   Max.   :47.00  
 (Other)        :1392                                                
      hour           month        date               weekend     holiday 
 Min.   : 0.00   Min.   :5   Min.   :2018-05-30   Min.   :0   Min.   :0  
 1st Qu.: 5.75   1st Qu.:5   1st Qu.:2018-05-30   1st Qu.:0   1st Qu.:0  
 Median :11.50   Median :5   Median :2018-05-30   Median :0   Median :0  
 Mean   :11.50   Mean   :5   Mean   :2018-05-30   Mean   :0   Mean   :0  
 3rd Qu.:17.25   3rd Qu.:5   3rd Qu.:2018-05-31   3rd Qu.:0   3rd Qu.:

min_date,max_date
2018-05-30,2018-05-31 23:00:00


In [57]:
head(bj_future_data, 20)

stationId,utc_time,id,hour,month,date,weekend,holiday,longitude,latitude,PM2.5,PM10,O3,test_id
aotizhongxin_aq,2018-05-30 00:00:00,0,0,5,2018-05-30,0,0,116.397,39.982,48.87392,80.88440,75.65979,aotizhongxin_aq#0
aotizhongxin_aq,2018-05-30 01:00:00,1,1,5,2018-05-30,0,0,116.397,39.982,48.92463,80.86317,73.92367,aotizhongxin_aq#1
aotizhongxin_aq,2018-05-30 02:00:00,2,2,5,2018-05-30,0,0,116.397,39.982,48.97534,80.84193,72.18755,aotizhongxin_aq#2
aotizhongxin_aq,2018-05-30 03:00:00,3,3,5,2018-05-30,0,0,116.397,39.982,49.02605,80.82070,70.45143,aotizhongxin_aq#3
aotizhongxin_aq,2018-05-30 04:00:00,4,4,5,2018-05-30,0,0,116.397,39.982,49.07676,80.79947,68.71531,aotizhongxin_aq#4
aotizhongxin_aq,2018-05-30 05:00:00,5,5,5,2018-05-30,0,0,116.397,39.982,49.12747,80.77824,66.97920,aotizhongxin_aq#5
aotizhongxin_aq,2018-05-30 06:00:00,6,6,5,2018-05-30,0,0,116.397,39.982,49.17818,80.75700,65.24308,aotizhongxin_aq#6
aotizhongxin_aq,2018-05-30 07:00:00,7,7,5,2018-05-30,0,0,116.397,39.982,49.22889,80.73577,63.50696,aotizhongxin_aq#7
aotizhongxin_aq,2018-05-30 08:00:00,8,8,5,2018-05-30,0,0,116.397,39.982,49.27960,80.71454,61.77084,aotizhongxin_aq#8
aotizhongxin_aq,2018-05-30 09:00:00,9,9,5,2018-05-30,0,0,116.397,39.982,49.33031,80.69331,60.03473,aotizhongxin_aq#9


In [ ]:
write.csv(bj_future_data[,c("test_id", "PM2.5", "PM10", "O3")], file = paste("bj_submission", Sys.Date(),".csv"), row.names = FALSE)

In [ ]:
# resetting the timezone
Sys.unsetenv("TZ")

In [ ]:
rm("bj_future_data")
rm("bj_lat_long_data")
rm("bj_aq_gm_combined_data")